In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import packages

In [2]:
!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en
!pip install bert-extractive-summarizer --upgrade --force-reinstall
# !pip install spacy==2.1.3 --upgrade --force-reinstall
!pip install spacy
!pip install -U nltk
!pip install pywsd==1.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-avuyku19
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-avuyku19
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bert_extractive_summarizer-0.10.1-py3-none-any.whl (25 kB)
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('popular')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Down

True





## BERT Extractive Summarizer
Summarize the text using BERT extractive summarizer. This is used to find important sentences and useful sentences from the complete text.

In [4]:
!pip install sacremoses
from summarizer import Summarizer

f = open("/content/drive/MyDrive/AutomaticMCQGenerator/full_text.txt","r")
full_text = f.read()

model = Summarizer()
result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)

summarized_text = ''.join(result)
print (summarized_text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Nile River fed Egyptian civilization for hundreds of years. It begins near the equator in Africa and flows north to the Mediterranean Sea. This soil was fertile, which means it was good for growing crops. Red Land, Black Land The ancient Egyptians lived in narrow bands of land on each side of the Nile. They called this region the black land because of the fertile soil that the floods deposited. The red land was the barren desert beyond the fertile region. Isolation The harsh desert acted as a barrier to keep out enemies. When the birds arrived, the annual flood waters would soon follow. Working together, they dug irrigation canals that carried river water to dry areas. These innovative, or new, techniques gave them more farmland. They were the first to grind wheat into flour and to mix the flour with yeast and water to make dough rise into bread. Egyptians often painted walls white to reflect the blazing heat. Poorer Egyptians simply went to the roof to cool off after sunset. They 

## Keyword Extraction
Get important keywords from the text and filter those keywords that are present in the summarized text.

In [5]:
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords

def get_nouns_multipartite(text):
    out=[]

    extractor = pke.unsupervised.MultipartiteRank()
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.load_document(input=text,stoplist=stoplist)
    #    not contain punctuation marks or stopwords as candidates.
    pos = {'PROPN'}
    #pos = {'VERB', 'ADJ', 'NOUN'}
    extractor.candidate_selection(pos=pos)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=20)

    for key in keyphrases:
        out.append(key[0])

    return out

keywords = get_nouns_multipartite(full_text) 
print (keywords)
filtered_keys=[]
for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
        filtered_keys.append(keyword)
        
print (filtered_keys)

['nile river', 'egyptians', 'egypt', 'nile', 'euphrates', 'tigris', 'old kingdom', 'crown', 'red land', 'longest river', 'narmer united upper', 'africa', 'mediterranean sea', 'hyksos', 'black land', 'new kingdom', 'ethiopia', 'middle kingdom', 'legend', 'lower egypt']
['nile river', 'egyptians', 'egypt', 'nile', 'old kingdom', 'crown', 'red land', 'narmer united upper', 'africa', 'mediterranean sea', 'black land', 'new kingdom', 'middle kingdom', 'legend', 'lower egypt']


## Sentence Mapping
For each keyword get the sentences from the summarized text containing that keyword. 

In [6]:
!pip install flashtext
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
        
print (keyword_sentence_mapping)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
{'nile river': ['The Nile River fed Egyptian civilization for hundreds of years.'], 'egyptians': ['Egyptians cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.', 'The Nile provided so well for Egyptians that sometimes they had surpluses, or more goods than they needed.', 'For example, some ancient Egyptians learned to be scribes, people whose job was to write and keep records.', 'Red Land, Black Land The ancient Egyptians lived in narrow bands of land on each side of the Nile.', 'This prosperity made life easier and provided greater opportunities for many Egyptians.', 'Egyptians created a government that divided the empire into 42 provinces.', 'Early Egyptians created a hieroglyphic system with about 700 characters.', 'Eventually, Egyptians equipped their reed boats with sails and oars.', 'Poorer Egyptians simply went to the roof to cool

## Generate MCQ
Get distractors (wrong answer choices) from Wordnet/Conceptnet and generate MCQ Questions.

In [7]:
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn

# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

#############################################################################
NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. 
#############################################################################


1)  _______  cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.
	 a )   Angolan
	 b )   Egyptians
	 c )   Algerian
	 d )   Bantu

More options:  ['Basotho', 'Beninese', 'Berber', 'Black African', 'Burundian', 'Cameroonian', 'Carthaginian', 'Chadian', 'Chewa', 'Congolese', 'Djiboutian', 'Egyptian', 'Ethiopian', 'Eurafrican', 'Ewe', 'Fulani'] 


2) As in many ancient societies, much of the knowledge of  _______  came about as priests studied the world to find ways to please the gods.
	 a )   Egypt
	 b )   Kuwait
	 c )   Iraq
	 d )   Saudi Arabia

More options:  ['Jordan', 'Israel', 'Fertile Crescent', 'Turkey', 'Iran', 'Lebanon', 'Shari', 'Maur

# Changing sentence to question

Using a T5 transformer, sentence and keyword is changed to question.

In [8]:
from transformers import T5ForConditionalGeneration,T5Tokenizer
!pip install sentencepiece==0.1.94

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96


In [9]:
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('t5-base')

MAX_SEQ_LENGTH = 128

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [23]:
def get_question(sentence,answer):
  sentence = " ".join(sentence.split()) 
  text = "context: {} answer: {}".format(sentence,answer)
  max_len = 256
  encoding = question_tokenizer.encode_plus(text,max_length=max_len, padding='max_length', return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = question_model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=200)


  dec = [question_tokenizer.decode(ids) for ids in outs]
  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

In [29]:
q = get_question("I love the color red.","red")
q = q.replace("<pad>"," ")
q = q.replace("</s>"," ")
q = q.strip()
print(q)

What color do I love?


In [30]:
index = 1
print ("#############################################################################")
print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
print ("#############################################################################\n\n")
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    # output = pattern.sub( " _______ ", sentence)
    output = get_question(sentence.lower(),each.lower())
    output = output.replace("<pad>"," ")
    output = output.replace("</s>"," ")
    output = output.strip()
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1
    

#############################################################################
NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. 
#############################################################################


1) Who cut the stems into strips?
	 a )   Bantu
	 b )   Algerian
	 c )   Angolan
	 d )   Egyptians

More options:  ['Basotho', 'Beninese', 'Berber', 'Black African', 'Burundian', 'Cameroonian', 'Carthaginian', 'Chadian', 'Chewa', 'Congolese', 'Djiboutian', 'Egyptian', 'Ethiopian', 'Eurafrican', 'Ewe', 'Fulani'] 


2) Priests studied the world to find ways to please the gods in what ancient society?
	 a )   Kuwait
	 b )   Egypt
	 c )   Saudi Arabia
	 d )   Iraq

More options:  ['Jordan', 'Israel', 'Fertile Crescent', 'Turkey', 'Iran', 'Lebanon', 'Shari', 'Mauritania', 'Nigeria', 'Somali peninsula', 'Sierra Leone', 'Malawi', 'North Africa', 'Senegal', 'Mozambique', 'Lake Tanganyika'] 


3) What 

# Deploy using Gradio

In [34]:
def process_gradio(full_text):
  #Summarize
  result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)
  summarized_text = ''.join(result)
  #Get nouns
  keywords = get_nouns_multipartite(full_text) 
  filtered_keys=[]
  for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
      filtered_keys.append(keyword)
  sentences = tokenize_sentences(summarized_text)
  keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
  #Get distractors
  key_distractor_list = {}
  for keyword in keyword_sentence_mapping:
      wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
      if wordsense:
          distractors = get_distractors_wordnet(wordsense,keyword)
          if len(distractors) ==0:
              distractors = get_distractors_conceptnet(keyword)
          if len(distractors) != 0:
              key_distractor_list[keyword] = distractors
      else:        
          distractors = get_distractors_conceptnet(keyword)
          if len(distractors) != 0:
              key_distractor_list[keyword] = distractors

  index = 1
  print ("#############################################################################")
  print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
  print ("#############################################################################\n\n")
  out = ""
  for each in key_distractor_list:
      sentence = keyword_sentence_mapping[each][0]
      pattern = re.compile(each, re.IGNORECASE)
      # output = pattern.sub( " _______ ", sentence)
      output = get_question(sentence.lower(),each.lower())
      output = output.replace("<pad>"," ")
      output = output.replace("</s>"," ")
      output = output.strip()
      # print ("%s)"%(index),output)
      out = out + "\n\n\n"+ str(index)+". "+output+"\n"
      choices = [each.capitalize()] + key_distractor_list[each]
      top4choices = choices[:4]
      random.shuffle(top4choices)
      optionchoices = ['a','b','c','d']
      curr_choices = ""
      for idx,choice in enumerate(top4choices):
          # print ("\t",optionchoices[idx],")"," ",choice)
          out = out + "\t"+optionchoices[idx]+") "+choice
      # print ("\nMore options: ", choices[4:20],"\n\n")
      index = index + 1
  return out

In [35]:
res = process_gradio(full_text)
print(res)

#############################################################################
NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. 
#############################################################################





1. Who cut the stems into strips?
	a) Algerian	b) Egyptians	c) Bantu	d) Angolan


2. Priests studied the world to find ways to please the gods in what ancient society?
	a) Iraq	b) Egypt	c) Kuwait	d) Saudi Arabia


3. What provided so well for egyptians that they had surpluses?
	a) Buganda	b) Nile	c) Entebbe	d) Gulu


4. What part of the upper egypt was white?
	a) Academic Degree	b) Aliyah	c) Crown	d) Academy Award


5. Where does the equator begin?
	a) Eurasia	b) Old World	c) Africa	d) Australia


6. What says narmer united upper and lower egypt?
	a) Adventure Story	b) Fable	c) Legend	d) Love Story


7. Which egypt had a red crown?
	a) Aswan	b) Eastern Desert	c) Lower egypt	d) Aswan High Da

In [36]:
!pip install gradio
import gradio as gr



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 2.3 MB 42.0 MB/s 
     |████████████████████████████████| 140 kB 55.5 MB/s 
     |████████████████████████████████| 272 kB 51.1 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 212 kB 60.3 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 271 kB 46.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 49.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 80 kB 8.0 MB/s 
     |████████████████████████████████| 68 kB 

In [ ]:
demo = gr.Interface(
    process_gradio,
    "text",
    "text"
)

In [37]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://34739.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7ff01c4c6110>,
 'http://127.0.0.1:7860/',
 'https://34739.gradio.app')